# DCNN test

DCNN test notebook is a place for exploring customized layers of dcnn with tensorflow.

In [1]:
import numpy as np
import tensorflow as tf

# from tensorflow.python import debug as tf_debug

*Do Experinemts on the padding function on tensorflow*

In [2]:
t = tf.constant([[1, 2, 3], [4, 5, 6]])

# Set up the padding values
# [1, 1]: Add 1 pad for "height(row)" dimension
# [2, 2]: Add 2 pads for "weight(col)" dimension
paddings = tf.constant([[1, 1,], [2, 2]])

t_pad = tf.pad(t, paddings, "CONSTANT")

with tf.Session() as sess:
    
    r = sess.run(t_pad)
    
    print(r)

[[0 0 0 0 0 0 0]
 [0 0 1 2 3 0 0]
 [0 0 4 5 6 0 0]
 [0 0 0 0 0 0 0]]


In [3]:
# Set up the padding values asymmetric 
# [1, 2]: Prepend 1 pad before the values and append 2 pads after the values in dimension 0
# [3, 4]: Prepend 3 pads befroe the values and append 4 pads after the values in dimension 1
paddings = tf.constant([[1, 2], [3, 4]])

t_pad = tf.pad(t, paddings, "CONSTANT")

with tf.Session() as sess:
    
    print(sess.run(t_pad))


[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 2 3 0 0 0 0]
 [0 0 0 4 5 6 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]


In [4]:
t_4d = tf.constant([[[[1, 2, 3], [4, 5, 6]]]])
paddings = tf.constant([[0,0], [0,0], [1, 1], [2, 2]])

t_pad = tf.pad(t_4d, paddings, "CONSTANT")

with tf.Session() as sess:
    
    r = sess.run(t_pad)
    print(r.shape)
    print(r)

(1, 1, 4, 7)
[[[[0 0 0 0 0 0 0]
   [0 0 1 2 3 0 0]
   [0 0 4 5 6 0 0]
   [0 0 0 0 0 0 0]]]]


## Test: KMaxPooling2D

KmaxPooling is a fundamental layer of dynamic cnn. 

#### Op implementation

In [5]:
import math

# 20180604 Lin, Y.D.: Read how tensorflow does padding dynamically. 
def _kmax_pooling2d(X, pool_size, strides, k, padding):
    
    h_pool = pool_size[0]
    w_pool = pool_size[1]

    h_stride = strides[0]
    w_stride = strides[1]
    
    h_pad = padding[0]
    w_pad = padding[1]
    
    h = X.shape[1]
    w = X.shape[2]
    
    X = np.reshape(X, (h, w, 1))
    
    # 20180613 LIN, Y.D. padding has been added in class KMaxPooling2D previosly. No Padding need here
    outputs_h = math.ceil((h-h_pool+1)/h_stride)
    outputs_w = k * math.ceil((w-w_pool+1)/w_stride)
    
    # 20180612 LIN, Y.D. Wait to solve boundary problem.
    outputs = np.zeros((outputs_h, outputs_w, 1), dtype=np.float64)

    m = 0
    for _h in range(0, h-h_pad, h_stride):
            
        n = 0
        for _w in range(0, w-w_pad, w_stride):
            
            flatten_slice = X[_h:_h+h_pool, _w:_w+w_pool].ravel()
            top_indices = np.sort(
                np.argpartition(flatten_slice, -1*np.arange(k))[h_pool+w_pool-k: h_pool+w_pool])
            outputs[m, n:n+k] = np.reshape(flatten_slice[top_indices], (1, k, 1))
            
            n += k
                
            if n >= outputs_w:
                break

        m += 1
        if m >= outputs_h:
            break
            
    return outputs

    # 20160618 LIN, Y.D. Tensorflow has some limit to get dynamic shape of input. Batch solution pending.
    # 20180612 LIN, Y.D. X is a 4-D tensor
#     outputs = []
#     for _X in X:
        
#         h = _X.shape[0]
#         w = _X.shape[1]
    
#         # 20180613 LIN, Y.D. padding has been added in class KMaxPooling2D previosly. No Padding need here
#         _outputs_h = math.ceil((h-h_pool+1)/h_stride)
#         _outputs_w = k * math.ceil((w-w_pool+1)/w_stride)
    
#         # 20180612 LIN, Y.D. Wait to solve boundary problem.
#         _outputs = np.zeros((_outputs_h, _outputs_w, 1), dtype=np.float64)
#         outputs.append(_outputs)
    
#     outputs = np.array(outputs)
    
    # 20180616 LIN, Y.D.: Batch cannot work so fine. (Pending)
#     batch = X.shape[0]
#     for _b in range(batch):
        
#         output_h = X[_b].shape[0]
#         output_w = X[_b].shape[1]
        
#         m = 0
#         for _h in range(0, h-h_pad, h_stride):
            
#             n = 0
#             for _w in range(0, w-w_pad, w_stride):
            
#                 flatten_slice = X[_b, _h:_h+h_pool, _w:_w+w_pool].ravel()
#                 top_indices = np.sort(
#                     np.argpartition(flatten_slice, -1*np.arange(k))[h_pool+w_pool-k: h_pool+w_pool])
#                 outputs[_b, m, n:n+k] = np.reshape(flatten_slice[top_indices], (1, k, 1))
            
#                 n += k
                
#                 if n >= output_w:
#                     break

#             m += 1
#             if m >= output_h:
#                 break
    
#     return outputs


#### Layer Implementation

In [92]:
#20180607 LIN, Y.D. 
class KMaxPooling2D(tf.layers.Layer):
    
    def __init__(self, pool_size, strides, k, padding='valid', name=None, **kwargs):
        
        super(KMaxPooling2D, self).__init__(name=name, **kwargs)
        
        if padding != 'same' and padding != 'valid':
            raise ValueError('Invalid Padding Options. Only SAME and VALID are valid.')
            
        # 20180610 LIN, Y.D. Add the exception when 
        if strides[0] == 0 and strides[1] == 0:
            raise ValueError('Invalid Stride Values: At least 1 stride value should be greater than 0.')

        self.k = k
        self.padding = padding
        self.pool_size = pool_size
        self.strides = strides
    
    def call(self, inputs):
        
        inputs_shape = inputs.get_shape().as_list()
        
        if len(inputs_shape) < 4:
            raise ValueError('The input should be a 4-D tensor')
        
        h = inputs_shape[1]
        w = inputs_shape[2]
        
        if inputs_shape == [None, None, None, None]:
            return [tf.constant([0]), tf.constant([0]), tf.constant([0]), tf.constant([0])]
        
        h_stride = self.strides[0]
        w_stride = self.strides[1]
        
        h_pool_s = self.pool_size[0]
        w_pool_s = self.pool_size[1]
        
        # 20180612 LIN, Y.D. 
        pool_size = tf.constant(self.pool_size, dtype=tf.int32)
        strides   = tf.constant(self.strides, dtype=tf.int32)
        k = tf.constant(self.k, dtype=tf.int32)
        padding = tf.constant([0, 0], dtype=tf.int32)
        
        if self.padding == 'valid':
            if ((h - h_pool_s) / h_stride + 1) % 1 != 0:
                raise ValueError('The height is invalid')
            if ((w - w_pool_s) / w_stride + 1) % 1 != 0:
                raise ValueError('The weight is invalid')
        
        elif self.padding == 'same':
            
            h_mod = h % h_stride
            w_mod = w % w_stride
        
            if h_mod == 0:
                h_pad = max([h_pool_s - h_stride, 0])
            else:
                h_pad = max([h_pool_s - h_mod, 0])
            
            if w_mod == 0:
                w_pad = max([w_pool_s - w_stride, 0])
            else:
                w_pad = max([w_pool_s - w_mod, 0])
        
            # 20180606 Y.D. Padding input with 0.
            h_pad_mod_2 = h_pad % 2
            w_pad_mod_2 = w_pad % 2
        
            if h_pad_mod_2 == 0:
                h_pad_top, h_pad_bottom = h_pad//2, h_pad//2
            else:
                h_pad_top, h_pad_bottom = h_pad//2, h_pad//2 + h_pad_mod_2
    
            if w_pad_mod_2 == 0:
                w_pad_left, w_pad_right = w_pad//2, w_pad//2
            else:
                w_pad_left, w_pad_right = w_pad//2, w_pad//2 + w_pad_mod_2
            
            pad_vals = tf.constant(
                [[0, 0], [h_pad_top, h_pad_bottom], [w_pad_left, w_pad_right], [0, 0]], 
                dtype=tf.int32)
        
            inputs = tf.pad(inputs, pad_vals, "CONSTANT")
            
            # 20180612 LIN, Y.D. Update inputs
            padding = tf.constant([h_pad_top+h_pad_bottom, w_pad_left+w_pad_right], dtype=tf.int32)
        
        
        outputs = tf.py_func(_kmax_pooling2d, [inputs, pool_size, strides, k, padding], tf.float64)
        
        return outputs
    

In [93]:
def kmax_pooling2d(inputs, pool_size, strides, k, padding, name=None):
    return KMaxPooling2D(pool_size, strides, k, padding=padding, name=name).apply(inputs)

In [95]:
# 20180605 LIN, Y.D.
with tf.Session() as sess:
    
    test_x = np.array([[[1,1,3], [3,1,2]]], dtype=np.float64)
    test_input = tf.placeholder(tf.float64, (1, 2, 3))
    test_reshape = tf.reshape(test_input, shape=(1,2,3,1))
    
    # Test Case 1: 
    test_layer = kmax_pooling2d(test_reshape, [2, 2], [1, 1], 2, 'same')
    res = sess.run(test_layer, feed_dict={ test_input: test_x })
    print(res)
    print(res.shape)
    print('-' * 10)
    
    # Test Case 2: Test if exception work.
    try:
        test_layer_2 = kmax_pooling2d(test_reshape, [3, 3], [0, 0], 2, 'same')
        res = sess.run(test_layer_2, feed_dict={test_input: test_x })
        print(res)
        print(res.shape)
    except ValueError as e:
        print('Error is caught successfully')
        print(e)
        pass
    print('-' * 10)
    
    # Test Case 3: Test Valid
    test_x_3 = np.array([[[1, 2, 3], [3, 1, 2], [2, 1, 3]]], dtype=np.float64)
    test_input_3 = tf.placeholder(tf.float64, (1, 3, 3))
    test_reshape_3 = tf.reshape(test_input_3, (1, 3, 3, 1))
    test_layer_3 = kmax_pooling2d(test_reshape_3, [2, 2], [1, 1], 2, 'valid')
    res = sess.run(test_layer_3, feed_dict={ test_input_3: test_x_3 })
    
    print(res)
    print(res.shape)
    print('-' * 10)
    
    # Test Case 4: If the first stride will cross over the border.
    test_layer_4 = kmax_pooling2d(test_reshape_3, [3, 3], [3, 3], 2, 'same')
    res = sess.run(test_layer_4, feed_dict={ test_input_3: test_x_3 })
    
    print(res)
    print(res.shape)
    print('-' * 10)
    
    # Test Case 5: Continue the previous test, we use valid as padding policy
    test_layer_5 = kmax_pooling2d(test_reshape_3, [3, 3], [3, 3], 2, 'valid')
    res = sess.run(test_layer_5, feed_dict={ test_input_3: test_x_3 })
    
    print(res)
    print(res.shape)
    print('-' * 10)
    
    # Test Case 6: Filter should not bigger than the input matrix
    test_layer_6 = kmax_pooling2d(test_reshape_3, [4, 4], [2, 2], 2, 'same')
    res = sess.run(test_layer_6, feed_dict={ test_input_3: test_x_3 })
    
    print(res)
    print(res.shape)
    print('-' * 10)
    
    # Test Case 7:  
    try:
        test_layer_7 = kmax_pooling2d(test_reshape_3, [4, 4], [2, 2], 2, 'valid')
        res = sess.run(test_layer_7, feed_dict={ test_input_3: test_x_3 })
        print(res)
        print(res.shape)
        print('-' * 10)
    except ValueError as e:
        print(e)
        print('Error is caught successfully')
        

[[[3.]
  [1.]
  [3.]
  [2.]
  [3.]
  [2.]]

 [[3.]
  [1.]
  [1.]
  [2.]
  [2.]
  [0.]]]
(2, 6, 1)
----------
Error is caught successfully
Invalid Stride Values: At least 1 stride value should be greater than 0.
----------
[[[2.]
  [3.]
  [3.]
  [2.]]

 [[3.]
  [2.]
  [2.]
  [3.]]]
(2, 4, 1)
----------
[[[1.]
  [2.]]]
(1, 2, 1)
----------
[[[1.]
  [2.]]]
(1, 2, 1)
----------
[[[2.]
  [3.]
  [0.]
  [0.]]

 [[1.]
  [0.]
  [0.]
  [0.]]]
(2, 4, 1)
----------
The height is invalid
Error is caught successfully


#### Train a simple model with kmaxpooling


## Test: Folding

Folding is introduced before the last KMax layer.

## Test: Varied Length Input

Varied length of text as input.


## Test: Stack a "non-deep" dcnn (1)

We stack a shallow dcnn to make sure no bug exists in inference stage.


## Test: Stack a "non-deep" dcnn (2)

Train a "non-deep" dcnn with varied length as input.